In [ ]:
# ! pip install -q kaggle
# from google.colab import files
# files.upload()
# ! mkdir ~/.kaggle
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


In [ ]:
import pandas as pd
import io
data=pd.read_csv('train.csv')
test=pd.read_csv('test_with_solutions.csv')

In [ ]:
data.head()


In [ ]:
test.head()

,Insult,Date,Comment,Usage
0,0,NaN,"""THE DRUDGE REPORT\\n\\n\\n\\nYou won't see th...",PublicTest
1,0,20120618222256Z,"""@ian21\xa0""Roger Clemens is the fucking man, ...",PublicTest
2,1,20120618213617Z,"""Agree with Alan you are an extremest idiot. ...",PublicTest
3,0,NaN,"""Really?\\n\\nI see Marc Lamont Hill on variou...",PrivateTest
4,0,20120620003825Z,"""Really suck isn't the word, when many of our ...",PrivateTest


In [ ]:
len(test)

2647

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
stem = PorterStemmer()
lem = WordNetLemmatizer()

def word_stemmer(words):
    stem_words = [stem.stem(o) for o in words]
    return " ".join(stem_words)
def word_lemmatizer(words):
   lemma_words = [lem.lemmatize(o) for o in words]
   return " ".join(lemma_words)

In [ ]:
i=3946
while(i>-1):
  data["Comment"][i]=word_stemmer(data["Comment"][i].split(" "))
  i=i-1
i=2647-1
while(i>-1):
  test["Comment"][i]=word_stemmer(test["Comment"][i].split(" "))
  i=i-1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = [data["Comment"][j] for j in range(0,3946+1)]
text2 = [test["Comment"][j] for j in range(0,2647)]

vectorizer = TfidfVectorizer()

vectorizer.fit(text)
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
x_train = vectorizer.transform([text[0]]+[text[i] for i in range(1,3946+1)] )
x_test = vectorizer.transform([text2[0]]+[text2[i] for i in range(1,2647)] )

print(x_train.toarray())

{'you': 14527, 'fuck': 4893, 'your': 14534, 'dad': 3086, 'realli': 10526, 'don': 3687, 'understand': 13393, 'point': 9828, 'it': 6362, 'seem': 11313, 'that': 12664, 'are': 900, 'mix': 7803, 'appl': 859, 'and': 739, 'oranges': 9208, 'major': 7370, 'of': 9094, 'canadian': 2051, 'can': 2049, 'ha': 5384, 'been': 1386, 'wrong': 14170, 'befor': 1393, 'now': 8740, 'will': 14025, 'be': 1340, 'again': 545, 'nunless': 8941, 'support': 12350, 'the': 12666, 'idea': 5935, 'noth': 8725, 'is': 6337, 'full': 4941, 'proof': 10163, 'or': 9206, 'perfect': 9576, 'so': 11800, 'take': 12482, 'chanc': 2237, 'if': 5956, 'we': 13854, 'should': 11537, 'inadvert': 6055, 'kill': 6716, 'son': 11877, 'daughter': 3145, 'then': 12687, 'them': 12683, 'break': 1793, 'alway': 687, 'regard': 10618, 'as': 966, 'collater': 2542, 'damag': 3101, 'like': 7106, 'in': 6052, 'wartim': 13818, 'sorry': 11902, 'but': 1962, 'chequ': 2293, 'mail': 7359, 'listen': 7153, 'dont': 3702, 'wanna': 13798, 'get': 5079, 'marri': 7455, 'to': 1

In [ ]:
y_train=[data.Insult[i] for i in range(0,3946+1)]
y_test=[test.Insult[i] for i in range(0,2647)]

In [ ]:
from array import array
res = array("i", y_test) 

In [ ]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(x_train.toarray(),y_train)
pred = model.predict(x_test.toarray())

In [ ]:
from sklearn.metrics import classification_report
print("Classification report for Naive Bayes on our dataset\n")
print(classification_report(pred,y_test))

Classification report for Naive Bayes on our dataset

              precision    recall  f1-score   support

           0       0.72      0.81      0.76      1726
           1       0.53      0.40      0.45       921

    accuracy                           0.67      2647
   macro avg       0.62      0.60      0.61      2647
weighted avg       0.65      0.67      0.65      2647



In [ ]:
import tqdm
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dropout, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pickle

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from numpy import zeros
from random import shuffle
from random import seed
from matplotlib import pyplot

In [ ]:
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(14599,)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(x_train.toarray(),data.Insult,epochs=8,batch_size=100,validation_split=0.3)

Train on 2762 samples, validate on 1185 samples
Epoch 1/8
2762/2762 [==============================] - 1s 192us/step - loss: 0.6461 - accuracy: 0.7393 - val_loss: 0.6154 - val_accuracy: 0.7207
Epoch 2/8
2762/2762 [==============================] - 0s 176us/step - loss: 0.5780 - accuracy: 0.7400 - val_loss: 0.5750 - val_accuracy: 0.7207
Epoch 3/8
2762/2762 [==============================] - 0s 166us/step - loss: 0.5313 - accuracy: 0.7400 - val_loss: 0.5497 - val_accuracy: 0.7207
Epoch 4/8
2762/2762 [==============================] - 0s 170us/step - loss: 0.4945 - accuracy: 0.7400 - val_loss: 0.5305 - val_accuracy: 0.7207
Epoch 5/8
2762/2762 [==============================] - 0s 171us/step - loss: 0.4573 - accuracy: 0.7411 - val_loss: 0.5095 - val_accuracy: 0.7283
Epoch 6/8
2762/2762 [==============================] - 1s 188us/step - loss: 0.4207 - accuracy: 0.7600 - val_loss: 0.4921 - val_accuracy: 0.7477
Epoch 7/8
2762/2762 [==============================] - 0s 169us/step - loss: 0.385

In [ ]:
def plot_accuracy(history):
    pyplot.plot(history.history['acc'])
    pyplot.plot(history.history['val_acc'])
    pyplot.title('model accuracy')
    pyplot.ylabel('accuracy')
    pyplot.xlabel('epoch')
    pyplot.legend(['training', 'validation'], loc='lower right')
    pyplot.show()

def plot_loss(history):
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.title('model loss')
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['training', 'validation'], loc='upper right')
    pyplot.show()

# summarize history for accuracy
plot_accuracy(history)

# summarize history for loss
plot_loss(history)

KeyError: ignored

In [ ]:
pred=model.predict(x_test.toarray())
print(pred)

In [ ]:
pred.reshape(1,2647)

In [ ]:
my[0][1]

0.19063377380371094

In [ ]:
my_pred = []
for i in range(0,len(y_test)):
  if(my[0][i] > 0.5):
    my_pred.append(1)
  else:
    my_pred.append(0)   

In [ ]:
my_pred

In [ ]:
print("Classification report for Keras Sequential model on our dataset\n")
print(classification_report(my_pred, y_test))